In [29]:

"""
Exemplo simples: prever a letra seguinte em 'abcd...' usando Keras.
Precisamos do TensorFlow instalado: `pip install tensorflow` (versão 2.15+).
"""
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# ----- Gera dados toy: sequências 'abc' -> 'd', 'bcd' -> 'e', etc. -----
alphabet = "abcdefghijklmnopqrstuvwxyz"
seq_length = 3

def make_dataset(alphabet, seq_length):
    X, y = [], []
    for i in range(len(alphabet) - seq_length):
        seq_in  = alphabet[i:i+seq_length]
        seq_out = alphabet[i+seq_length]
        # one‑hot para letras
        X.append([ord(c) - ord('a') for c in seq_in])
        y.append(ord(seq_out) - ord('a'))
    X = to_categorical(X, num_classes=len(alphabet))  # shape: (samples, seq_len, 26)
    y = to_categorical(y, num_classes=len(alphabet))  # shape: (samples, 26)
    return X, y

X, y = make_dataset(alphabet, seq_length)
print("Shape X:", X.shape, "Shape y:", y.shape)  # (23, 3, 26) e (23, 26)

# ----------------------- Modelo LSTM simples ---------------------------
model = Sequential([
    LSTM(32, input_shape=(seq_length, len(alphabet))),
    Dense(len(alphabet), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ---------------------- Treinamento relâmpago --------------------------
model.fit(X, y, epochs=300, verbose=0)

# ----------------------- Teste: prever 'def' -> ? ----------------------
test_seq = "def"
test_in  = to_categorical([[ord(c) - ord('a') for c in test_seq]], 26)
pred_idx = model.predict(test_in)[0].argmax()
print(f"A rede acha que depois de '{test_seq}' vem '{chr(pred_idx + ord('a'))}'")


Shape X: (23, 3, 26) Shape y: (23, 26)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 32)                  │           7,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 26)                  │             858 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,410 (32.85 KB)

 Trainable params: 8,410 (32.85 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
A rede acha que depois de 'def' vem 'g'


In [ ]:
import numpy as np

# --------------------- Hipóteses do "mini‑problema" --------------------

#Os números são o “passo a passo” de um LSTM feito à mão: cada linha mostra como a
#célula decide o que esquecer, o que entrar na memória e o que sai.

np.random.seed(42)
seq_len      = 3   # 3 passos de tempo (ex.: três palavras)
input_dim    = 4   # tamanho do vetor que representa cada palavra
hidden_dim   = 2   # quantos neurônios na célula LSTM

# ------------- Pesos (aleatórios) para demonstrar o cálculo ------------
def glorot(shape):
    limit = np.sqrt(6 / sum(shape))
    return np.random.uniform(-limit, limit, shape)

W_f = glorot((hidden_dim, input_dim))   # pesos da porta de esquecer
U_f = glorot((hidden_dim, hidden_dim))
b_f = np.zeros((hidden_dim, 1))

W_i = glorot((hidden_dim, input_dim))   # pesos da porta de entrada
U_i = glorot((hidden_dim, hidden_dim))
b_i = np.zeros((hidden_dim, 1))

W_c = glorot((hidden_dim, input_dim))   # pesos do vetor candidato
U_c = glorot((hidden_dim, hidden_dim))
b_c = np.zeros((hidden_dim, 1))

W_o = glorot((hidden_dim, input_dim))   # pesos da porta de saída
U_o = glorot((hidden_dim, hidden_dim))
b_o = np.zeros((hidden_dim, 1))

# ----------------------- Ativação e utilitários ------------------------
sigmoid = lambda x: 1 / (1 + np.exp(-x))

# --------------------------- Dados de entrada --------------------------
X = np.random.randn(seq_len, input_dim, 1)  # (t, features, batch=1)

# -------------------- Vetores de estado inicial ------------------------
h_prev = np.zeros((hidden_dim, 1))  # saída anterior
c_prev = np.zeros((hidden_dim, 1))  # memória anterior

print("=== Começo da simulação ===")
for t in range(seq_len):
    x_t = X[t]

    # 1️⃣ Porta de Esquecer (forget gate)
    f_t = sigmoid(W_f @ x_t + U_f @ h_prev + b_f)

    # 2️⃣ Porta de Entrada (input gate)
    i_t = sigmoid(W_i @ x_t + U_i @ h_prev + b_i)

    # 3️⃣ Vetor Candidato (valores novos que *podem* entrar na memória)
    ĉ_t = np.tanh(W_c @ x_t + U_c @ h_prev + b_c)

    # 4️⃣ Atualização da memória
    c_t = f_t * c_prev + i_t * ĉ_t

    # 5️⃣ Porta de Saída
    o_t = sigmoid(W_o @ x_t + U_o @ h_prev + b_o)

    # 6️⃣ Saída final do bloco LSTM
    h_t = o_t * np.tanh(c_t)

    # ---------- Impressões pedagógicas ----------
    print(f"\nPasso t={t}")
    print("x_t:\n", x_t.ravel())
    print("f_t (esquece):", f_t.ravel())
    print("i_t (entrada):", i_t.ravel())
    print("ĉ_t (candidato):", ĉ_t.ravel())
    print("c_t (memória):", c_t.ravel())
    print("o_t (saída gate):", o_t.ravel())
    print("h_t (saída bloco):", h_t.ravel())

    # 7️⃣ Propaga para o próximo tempo
    h_prev, c_prev = h_t, c_t

print("\n", h_t)

=== Começo da simulação ===

Passo t=0
x_t:
 [-1.32818605  0.19686124  0.73846658  0.17136828]
f_t (esquece): [0.70830745 0.56245234]
i_t (entrada): [0.17153575 0.5884803 ]
ĉ_t (candidato): [-0.20668741 -0.36109718]
c_t (memória): [-0.03545428 -0.21249857]
o_t (saída gate): [0.6484427  0.61176091]
h_t (saída bloco): [-0.02298044 -0.12807631]

Passo t=1
x_t:
 [-0.11564828 -0.3011037  -1.47852199 -0.71984421]
f_t (esquece): [0.24205837 0.72028517]
i_t (entrada): [0.83199007 0.64240125]
ĉ_t (candidato): [0.52965735 0.27763894]
c_t (memória): [0.43208765 0.02529603]
o_t (saída gate): [0.44804946 0.70727141]
h_t (saída bloco): [0.18238503 0.01788734]

Passo t=2
x_t:
 [-0.46063877  1.05712223  0.34361829 -1.76304016]
f_t (esquece): [0.71795705 0.09985228]
i_t (entrada): [0.50434914 0.69494626]
ĉ_t (candidato): [0.2090436  0.39114117]
c_t (memória): [0.41565134 0.27434796]
o_t (saída gate): [0.39326225 0.18115969]
h_t (saída bloco): [0.15465455 0.04849028]

 [[0.15465455]
 [0.04849028]]
